In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
# Load rain and hotel occupancy data
r16 = pd.read_csv("data/rain2016.csv")
r17 = pd.read_csv("data/rain2017.csv")
r20 = pd.read_csv("data/rain2020.csv")
R=pd.concat([r16,r17,r20])[['Date', 'Rain']].sort_values('Date')
H = pd.read_csv("data/hotel-nights.csv")
H['per_day'] = H['Rooms']/H['Days']
R['rooms']=0
for i, row in R.iterrows(): 
    R.at[i,"rooms"]=H.loc[ H['Month']==row["Date"][:7] ,'per_day'].values[0]

In [ ]:
ds16 = pd.read_csv("data/2016_Data.csv").sort_values(['Date', 'Location'])
ds16['medium']='easy_gel'
ds17 = pd.read_csv("data/2017_Data.csv").sort_values(['Date', 'Location'])
ds20 = pd.read_csv("data/2020_Data.csv").query("medium=='levine'").sort_values(['Date', 'Location', 'Plate_number'])

In [ ]:
def expand_columns(ds, n=3):
    plate_prefix = [f"P{i+1}_" for i in range(n)]
    X=pd.concat([ds]*n) 
    X['Plate_number']=np.tile(np.arange(1,n+1),len(ds))
    plates=[s for s in ds.columns.tolist() if any(xs in s for xs in plate_prefix)]
    for p in plates:
        colname=p[3:] 
        X.loc[X['Plate_number'] == int(p[1]) , colname] =   ds[p]
    return X

def map_columns(ds, col_dict):
    col_dict={**common_cols, **col_dict}
    x=ds.rename(columns=col_dict)
    return x[list(col_dict.values())]

def calcRain(w):
    i=R.loc[R['Date']==w].index.values[0]
    r=R.iloc[i-2:i].agg('sum')['Rain']
    h=R.iloc[i-6:i].agg('sum')['rooms']
    return (h,r)

def annotate_weeks(ds, weeks):
    ds['week']=0
    ds['festival']=''
    ds['rain']=0
    ds['rooms']=0
    for w, i in zip(weeks, range(len(weeks))):
        I=ds['Date']==w
        ds.loc[I, 'week']= i+1
        ds.loc[I, 'festival']= weeks[w]
        h, r = calcRain(w)
        ds.loc[I, 'rain'] = r
        ds.loc[I, 'rooms']= h
    return ds

def prepare_year(ds, col_dict, weeks, locations, medium):
    x=map_columns(ds, col_dict)
    return annotate_weeks(x, weeks) \
            .query(' or '.join([ f"(Location=='{u}')" for u in locations])) \
            .query(f"medium==\'{medium}\'").drop(columns=['medium'])
    
sites=['VNX','MRD','SVT']
common_cols={'Date' : 'Date',
                'Location' : 'Location' ,
                'Plate_number' : 'Plate_number',
                'medium' : 'medium'}

In [ ]:
# 2016
weeks16= {'2016-06-21' : 'before',
            '2016-06-28' : 'before',
            '2016-07-05' : 'during',
            '2016-07-12' : 'during',
            '2016-07-19' : 'after',
            '2016-07-26' : 'after',
            '2016-08-02' : 'after',
            '2016-08-09' : 'after'} 
ds16['P2_qty_sample'] = ds16['P1_qty_sample']
ds16['P3_qty_sample'] = ds16['P1_qty_sample']
Y16 = prepare_year(expand_columns(ds16,3), {
                '24h_big_blue' : 'Bioindicator',
                '24h_med_blue' : 'Coliform',
                'qty_sample' : 'volume'
            }, weeks16, sites, 'easy_gel')
#Y16.to_csv("Y16.csv")


In [ ]:
# 2017
weeks17 = {'2017-06-12' : 'before',
    '2017-06-19' : 'before',
    '2017-06-26' : 'before',
    '2017-07-03' : 'during',
    '2017-07-10' : 'during',
    '2017-07-17' : 'after',
    '2017-07-24' : 'after',
    '2017-07-31' : 'after',}

Y17 = prepare_year(expand_columns(ds17,3), {
            'Date' : 'Date',
            'Location' : 'Location' ,
            'Plate_number' : 'Plate_number',
            '24h_big_blue' : 'Bioindicator',
            'fluo_halo_colonies' : 'Bioindicator UV+',
            '24h_med_blue' : 'Coliform',
            'qty_sample' : 'volume'
        }, weeks17, sites, 'easy_gel')
#Y17.to_csv("Y17.csv")

In [ ]:
# 2020
weeks20 = {'2020-06-11' : 'before',
                            '2020-06-18' : 'before',
                            '2020-06-25' : 'before',
                            '2020-07-01' : 'before',
                            '2020-07-09' : 'before',
                            '2020-07-16' : 'before',
                            '2020-07-22' : 'before',
                            '2020-08-06' : 'before'}

Y20=prepare_year(ds20, { 'green_met' : 'Bioindicator',
                            'purple' : 'Coliform',
                            'volume' : 'volume'
                        }, weeks20 , sites, 'levine')
#Y20.to_csv("Y20.csv")

In [ ]:
D=pd.concat([Y16,Y17,Y20])
D.to_pickle("data/allyears.pkl")